In [35]:
import numpy as np
import pandas as pd
import json
import os
#from tabulate import tabulate

import matplotlib.pyplot as plt

In [36]:
import git
repo = git.Repo('.', search_parent_directories=True)
repo_path = repo.working_tree_dir

In [62]:
path1 = os.path.join(repo_path,r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\FilesAbsEnergy\10000")
path2 = os.path.join(repo_path,r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\FilesDE-AVQE\10000")
dataframes = {}
potentials = [f for f in os.listdir(path1) if os.path.isdir(os.path.join(path1, f))]
potentials = ['DW']

In [63]:
all_data = []
cutoff_list = [2,4,8,16,32,64]

for path in [path1,path2]:

    converged_runs = []
    mean_iters = []
    mean_evals = []
    min_e = []
    delta_min_e = []
    median_e = []
    delta_median_e = []
    mean_e = []
    delta_mean_e = []
    exact_e = []
    time_taken = []

    num_vqe = []
    shots_list = []
    maxiter = []
    cutoffs = []
    ps = []

    for potential in potentials:
        for cutoff in cutoff_list:

            d_path = os.path.join(path, potential, f'{potential}_{cutoff}.json')
        
            try:
                with open(d_path, 'r') as json_file:
                    data = json.load(json_file)
            

                converged_indices = [i for i, success in enumerate(data['success']) if success]

                ps.append(potential)
                cutoffs.append(cutoff)

                num_vqe.append(data['num_VQE'])
                shots_list.append(data['shots'])

                optimzer = data['Optimizer']
                maxiter.append(optimzer['maxiter'])
                
                exact_min_e = np.min(data['exact_eigenvalues'])
                exact_e.append(exact_min_e)

                converged_runs.append(sum(data['success']))

                #iters = [data['num_iters'][i] for i in converged_indices]
                #mean_iters.append(int(np.round(np.mean(iters))))
                mean_iters.append(int(np.round(np.mean(data['num_iters']))))

                #evals = [data['num_evaluations'][i] for i in converged_indices]
                #mean_evals.append(int(np.round(np.mean(evals))))
                #mean_evals.append(int(np.round(np.mean(data['num_evaluations']))))

                #results = [data['results'][i] for i in converged_indices]
                results = data['results']
                min_e.append(np.min(results))
                delta_min_e.append(abs(exact_min_e - np.min(results)))
                median_e.append(np.median(results))
                delta_median_e.append(abs(exact_min_e - np.median(results)))
                mean_e.append(np.mean(results))
                delta_mean_e.append(abs(exact_min_e - np.mean(results)))

                #run_times = [data['run_times'][i] for i in converged_indices]
                #time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
                time_series = np.array([pd.Timedelta(t).total_seconds() for t in data['run_times']])
                mean_vqe_time = np.mean(time_series)
                #mean_vqe_time = pd.Timedelta(seconds=np.mean(time_series))
                time_taken.append(np.mean(mean_vqe_time))
                #time_taken.append(data['parallel_run_time'])

            except Exception as e:
                print(e)
                print("Unable to open file...")

    df = {
    'potential': ps,
    'cutoff': cutoffs,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'Mean Iter': mean_iters,
    #'Mean evals': mean_evals,
    'Mean VQE time (s)': [f"{t:.2f}" for t in time_taken],
    #'E_VQE': [f"{num:.5e}" for num in median_e],
    #'Exact': [f"{num:.5e}" for num in exact_e],
    #'Delta': [f"{num:.5e}" for num in delta_median_e],
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    #'VQE mean E': [f"{num:.5e}" for num in mean_e],
    'Delta mean E': [f"{num:.5e}" for num in delta_mean_e]
    
    }

    all_data.append(df.copy())

In [64]:
merged = pd.DataFrame(all_data[0]).merge(pd.DataFrame(all_data[1]).reset_index(), on=['potential', 'cutoff'], suffixes=('_path1', '_path2')).drop(columns=['index'])
merged


,potential,cutoff,Converged Runs_path1,Mean VQE time (s)_path1,Delta min E_path1,Delta median E_path1,Delta mean E_path1,Converged Runs_path2,Mean VQE time (s)_path2,Delta min E_path2,Delta median E_path2,Delta mean E_path2
0,DW,2,100/100,0.19,0.00000e+00,0.00000e+00,5.12843e-01,100/100,0.24,0.00000e+00,0.00000e+00,4.58891e-01
1,DW,4,100/100,0.82,5.97459e-02,2.11843e-01,3.83645e-01,100/100,1.90,6.01650e-02,1.45912e-01,3.55271e-01
2,DW,8,100/100,5.57,5.14859e-01,8.16557e-02,5.79831e-01,100/100,13.58,4.52211e-01,1.17044e-01,6.76229e-01
3,DW,16,100/100,16.45,8.91583e-01,4.11113e-01,9.58042e-01,100/100,49.76,2.19859e+00,6.17636e-02,6.83415e-01
4,DW,32,100/100,46.77,8.91623e-01,4.09390e-01,2.52612e+00,100/100,246.55,8.65670e+00,8.07421e-01,1.61323e+00
5,DW,64,100/100,147.51,8.91626e-01,8.90220e-01,2.82064e+00,100/100,1382.53,3.81980e+01,5.28039e+00,3.39229e+00


In [65]:
exclude = ['potential', 'cutoff', 'Converged Runs_path1','Converged Runs_path2']
cols_to_float = [c for c in merged.columns if c not in exclude]

merged[cols_to_float] = merged[cols_to_float].astype(float)


In [66]:
def compare_metrics(row, pairs, tol=1e-15):
    styles = []
    for pl_col, qk_col in pairs:
        pl_val, qk_val = row[pl_col], row[qk_col]

        if abs(pl_val - qk_val) <= tol:
            styles += ["background-color: #ffffcc", "background-color: #ffffcc"]
        elif pl_val < qk_val:
            styles += ["background-color: green", ""]
        else:
            styles += ["", "background-color: green"]
    return styles


In [67]:
metric_pairs = [("Mean VQE time (s)_path1", "Mean VQE time (s)_path2"), ("Delta min E_path1","Delta min E_path2"), ("Delta median E_path1","Delta median E_path2"), ("Delta mean E_path1","Delta mean E_path2")]
merged.style.apply(lambda r: compare_metrics(r, metric_pairs),
                        subset=[c for pair in metric_pairs for c in pair],
                        axis=1)

,potential,cutoff,Converged Runs_path1,Mean VQE time (s)_path1,Delta min E_path1,Delta median E_path1,Delta mean E_path1,Converged Runs_path2,Mean VQE time (s)_path2,Delta min E_path2,Delta median E_path2,Delta mean E_path2
0,DW,2,100/100,0.190000,0.000000,0.000000,0.512843,100/100,0.240000,0.000000,0.000000,0.458891
1,DW,4,100/100,0.820000,0.059746,0.211843,0.383645,100/100,1.900000,0.060165,0.145912,0.355271
2,DW,8,100/100,5.570000,0.514859,0.081656,0.579831,100/100,13.580000,0.452211,0.117044,0.676229
3,DW,16,100/100,16.450000,0.891583,0.411113,0.958042,100/100,49.760000,2.198590,0.061764,0.683415
4,DW,32,100/100,46.770000,0.891623,0.409390,2.526120,100/100,246.550000,8.656700,0.807421,1.613230
5,DW,64,100/100,147.510000,0.891626,0.890220,2.820640,100/100,1382.530000,38.198000,5.280390,3.392290
